In [1]:
import tensorflow as tf
from data_loader.load_dataset import PF_Pascal

In [2]:
pf_pascal = PF_Pascal("/home/files/datasets/PF-dataset-PASCAL")

In [3]:
ds = pf_pascal.dataset_loader("categorical_pair")

In [4]:
for i,j in ds.take(10):
    print(i.shape, j.shape)

(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
(200, 200, 3) (200, 200, 3)
